[View in Colaboratory](https://colab.research.google.com/github/tjwei/tensorflow-tutorial/blob/master/gan/Simple_GAN.ipynb)

In [0]:
import keras
from PIL import Image
import numpy as np

In [0]:
from keras.datasets import mnist
(train_X, train_y), (test_X, test_y) = mnist.load_data()

In [0]:
train_X = (train_X[..., None]-127.5)/128

In [0]:
from IPython.display import display
def showX(X, rows=1):
    assert X.shape[0] % rows == 0
    int_X = (X*128+128).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(rows, -1,28,28).swapaxes(1,2).reshape(28*rows,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:10])
print(train_y)

In [0]:
NZ = 32

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D, Reshape

netD = Sequential([
    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='selu', input_shape=(28,28,1)),
    Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation='selu'),
    Conv2D(filters=32, kernel_size=3, strides=2, padding='valid', activation="selu"),
    Conv2D(filters=1, kernel_size=3, strides=3, padding='valid', activation="sigmoid"),
    Reshape((1,)),
])

In [0]:
netD.summary()

In [0]:
from keras.layers import Conv2DTranspose, Reshape
netG = Sequential([
    Reshape( (1,1,NZ), input_shape=(NZ,)),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='valid', activation='selu'),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='valid', activation='selu'),
    Conv2DTranspose(filters=32, kernel_size=3, strides=2, padding='same', activation='selu'),
    Conv2DTranspose(filters=1, kernel_size=3, strides=2, padding='same', activation='tanh')
])

In [0]:
netG.summary()

In [0]:
from keras.models import Model
GD_output = netD(netG.outputs)
netGD = Model(inputs=netG.inputs, outputs=[GD_output])

In [0]:
netGD.summary()

In [0]:
import numpy as np
showX(netG.predict(np.random.normal( size=(10, NZ))))

In [0]:
import keras.backend as K
from keras.optimizers import RMSprop, SGD, Adam

#loss_fn = lambda output, target : K.mean(K.binary_crossentropy(output, target))
loss_fn = lambda output, target : -K.mean(K.log(output+1e-12)*target+K.log(1-output+1e-12)*(1-target))

input_real = netD.inputs[0]
input_z = netG.inputs[0]

output_D_real = netD.outputs[0]
output_D_fake = netGD.outputs[0]

loss_D_real = loss_fn(output_D_real, K.ones_like(output_D_real))
loss_D_fake = loss_fn(output_D_fake, K.zeros_like(output_D_fake))
loss_D = (loss_D_real +loss_D_fake)/2

training_updates = RMSprop(lr=1e-3).get_updates(netD.trainable_weights,[],loss_D)
netD_train = K.function([input_z, input_real],[loss_D/2], training_updates)






In [0]:
loss_G = loss_fn(output_D_fake, K.ones_like(output_D_fake))
training_updates = RMSprop(lr=1e-3).get_updates(netG.trainable_weights,[], loss_G)
netG_train = K.function([input_z], [loss_G], training_updates)

In [0]:
def minibatch(dataset, batchsize):
  while True:
    yield dataset[np.random.choice(dataset.shape[0], batchsize, replace=False)]


In [0]:
train_batch = minibatch(train_X, 10)
for i in range(5):
  X = next(train_batch)
  showX(X)

In [0]:
import time
from IPython.display import clear_output
t0 = time.time()
gen_iterations = 0
errG = 0
errG_sum = errD_sum = 0

display_iters = 100
batch_size = 32

train_batch = minibatch(train_X, batch_size)
fixed_Z = np.random.normal(scale=0.5, size=(100, NZ))
while gen_iterations < 10000: 
    X = next(train_batch)
    Z = np.random.normal(size=(batch_size, NZ))
    errD, = netD_train([Z, X])
    errD_sum +=errD
    
    #Z = np.random.normal(size=(batch_size, NZ))
    errG, = netG_train([Z])
    errG_sum += errG    
    gen_iterations+=1
    
    if gen_iterations%display_iters==0:
        clear_output(True)
        print('[%d] Loss_D: %f Loss_G: %f'
        % (gen_iterations, errD_sum/display_iters, errG_sum/display_iters), time.time()-t0)
        fakeX = netG.predict(fixed_Z)
        showX(fakeX, 10)
        errG_sum = errD_sum = 0